In [44]:
import folium
import geopandas as gpd
import pandas as pd
import branca.colormap as cm

# 1. Baca file GeoJSON
gdf = gpd.read_file("all_kabkota_ind.geojson")

# Kelompokkan berdasarkan provinsi dan gabungkan geometry
provinsi_gdf = gdf.dissolve(by='province_id', aggfunc='first')

# 2. Pilih kabupaten/kota target
wilayah_target = [
    "KEPULAUAN SERIBU", "KOTA JAKARTA UTARA", "KOTA JAKARTA TIMUR", "KOTA JAKARTA BARAT",
    "KOTA JAKARTA SELATAN", "KOTA JAKARTA PUSAT", "KOTA DEPOK", "KOTA BEKASI", "KOTA TANGERANG",
    "KOTA BOGOR", "KOTA SERANG", "SERANG", "BOGOR", "TANGERANG", "BEKASI", "DEPOK",
    "KOTA TANGERANG SELATAN", "KUDUS", "JEPARA", "KOTA SURAKARTA", "SUKOHARJO", "BOYOLALI",
    "KLATEN", "KARANGANYAR", "MAGETAN", "MADIUN", "KOTA MADIUN"
]
gdf = gdf[gdf['name'].isin(wilayah_target)]

# 3. Data radon (dummy)
data_radon = pd.DataFrame({
    'name': wilayah_target,
    'radon': [50] * 27
})

# 4. Gabungkan GeoDataFrame dengan data radon
gdf = gdf.merge(data_radon, on='name')

# 5. Buat colormap (misalnya gradasi dari biru → merah)
colormap = cm.LinearColormap(
    colors=['#d7191c', '#2c7bb6', '#ffffbf'],  # biru → kuning → merah
    vmin=gdf['radon'].min(),
    vmax=gdf['radon'].max(),
    caption="Konsentrasi Radon (Bq/m³)"
)

# 6. Buat peta dasar
m = folium.Map(location=[-7.5, 111], zoom_start=7, tiles="OpenStreetMap")

# Layer 2: Kabupaten dengan warna sesuai radon
folium.GeoJson(
    gdf,
    name="Radon",
    style_function=lambda x: {
        'fillColor': colormap(x['properties']['radon']),
        'color': 'gray',
        'weight': 2,
        'fillOpacity': 0.4
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name', 'radon'],
        aliases=["Wilayah:", "Radon (Bq/m³):"],
        sticky=True,
        style=(
            "background-color: white; "
            "color: black; "
            "font-weight: bold; "
            "font-size: 11px; "
            "border:1px solid gray; "
            "border-radius:3px; "
            "padding:3px;"
        )
    )
).add_to(m)

# Layer 1: Batas Provinsi (hitam, tebal)
folium.GeoJson(
    provinsi_gdf,
    name="Batas Provinsi",
    style_function=lambda x: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1,
        'opacity': 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['prov_name'],
        aliases=["Provinsi:"],
        sticky=True
    )
).add_to(m)



# Tambahkan legenda colormap
colormap.add_to(m)

# Simpan hasil
m.save("peta_radon_folium.html")
